In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [7]:
data = pd.read_csv('/content/LTCUSDT_data.csv, date_parser = True)
data

SyntaxError: ignored

In [5]:
data_training = data[data['Date']<'2019-12-25 23:00:00'].copy()
data_training

NameError: ignored

In [0]:
data_test = data[data['Date']>='2019-12-25 23:00:00'].copy()
data_test

In [0]:
training_data = data_training.drop(['Date'], axis = 1)
training_data.tail()

In [0]:
scaler = MinMaxScaler()
training_data = scaler.fit_transform(training_data)
training_data

In [0]:
X_train = []
y_train = []

In [0]:
training_data

In [0]:
for i in range(60, training_data.shape[0]):
  X_train.append(training_data[i-60:i])
  y_train.append(training_data[i, 0])

In [0]:
X_train, y_train = np.array(X_train), np.array(y_train)

In [0]:
X_train.shape, y_train.shape

In [0]:
#### Building LSTM

In [0]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

In [0]:
regressior = Sequential()

regressior.add(LSTM(units = 60, activation = 'relu', return_sequences = True, input_shape = (X_train.shape[1], 5)))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units = 60, activation = 'relu', return_sequences = True))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units = 80, activation = 'relu', return_sequences = True))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units = 120, activation = 'relu'))
regressior.add(Dropout(0.2))

regressior.add(Dense(units = 1))

In [0]:
regressior.summary()

In [0]:
regressior.compile(optimizer='adam', loss = 'mean_squared_error')

In [0]:
regressior.fit(X_train, y_train, epochs=1, batch_size=32)

In [0]:
### Prepare Test Dataset

In [0]:
data_test.head()

In [0]:
data_training.tail(60)

In [0]:
past_60_minutes = data_training.tail(60)

In [0]:
df = past_60_minutes.append(data_test, ignore_index = True)
df = df.drop(['Date'], axis = 1)
df.tail()

In [0]:
inputs = scaler.transform(df)
inputs.shape

In [0]:
X_test = []
y_test = []

for i in range(60, inputs.shape[0]):
  X_test.append(inputs[i-60:i])
  y_test.append(inputs[i, 0])

In [0]:
X_test, y_test = np.array(X_test), np.array(y_test)
X_test.shape

In [0]:
y_pred = regressior.predict(X_test)

In [0]:
scaler.scale_[0]
scale = 1/scaler.scale_[0]
scale
scale2 = 0.0000743612

In [0]:
y_pred = y_pred*scale
y_test = y_test*scale

In [0]:
### Visualization

In [0]:
plt.figure(figsize=(10,6))
plt.plot(y_test, color = 'red', label = 'Real LTC/USDT Price')
plt.plot(y_pred, color = 'blue', label = 'Predicted LTC/USDT Price')
plt.title('LTC/USDT Price Prediction')
plt.xlabel('Time')
plt.ylabel('LTC/USDT Price')
plt.legend()
plt.show()